# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,winnemucca,40.9730,-117.7357,10.25,48,0,3.63,US,1715273070
1,1,lihue,21.9789,-159.3672,20.77,86,75,2.06,US,1715272805
2,2,puerto natales,-51.7236,-72.4875,1.25,93,75,1.03,CL,1715273070
3,3,port mathurin,-19.6833,63.4167,25.61,75,100,11.24,MU,1715273070
4,4,bethel,41.3712,-73.4140,19.70,58,0,6.17,US,1715273071


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoEco",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df["Max Temp"] < 25) & 
    (city_data_df["Max Temp"] > 10) &
    (city_data_df["Wind Speed"] < 10) & 
    (city_data_df["Cloudiness"] == 0)
)

ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,winnemucca,40.9730,-117.7357,10.25,48,0,3.63,US,1715273070
4,4,bethel,41.3712,-73.4140,19.70,58,0,6.17,US,1715273071
20,20,dryden,49.7833,-92.7503,10.16,46,0,3.09,CA,1715272973
50,50,hermanus,-34.4187,19.2345,13.78,77,0,4.94,ZA,1715272879
68,68,bredasdorp,-34.5322,20.0403,13.51,77,0,2.48,ZA,1715273084


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,winnemucca,US,40.9730,-117.7357,48,
4,bethel,US,41.3712,-73.4140,58,
20,dryden,CA,49.7833,-92.7503,46,
50,hermanus,ZA,-34.4187,19.2345,77,
68,bredasdorp,ZA,-34.5322,20.0403,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
api_key = geoapify_key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lon},{lat},{radius}"
    bias = f"proximity:{lon},{lat}"
    
    params = {
        "categories": categories,
        "filter": filters,
        "bias": bias,
        "apiKey": api_key
    }
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
winnemucca - nearest hotel: The Overland Hotel
bethel - nearest hotel: Hampton Inn Danbury
dryden - nearest hotel: Holiday Inn Express
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: Victoria Hotel
lompoc - nearest hotel: Red Roof Inn Lompoc
kruisfontein - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
la plata - nearest hotel: Dazzler by Wyndham La Plata
shihezi - nearest hotel: 石河子宾馆
east london - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
brainerd - nearest hotel: Days Inn & Suites by Wyndham Baxter Brainerd Area
greenwood - nearest hotel: Hospitality Suites
katsuura - nearest hotel: 潮騒の宿
talcahuano - nearest hotel: Club Recreativo Armada de Chile
brookings - nearest hotel: Quality Inn
steinbach - nearest hotel: Alt Steinbach
rocha - nearest hotel: Hotel Municipal
port hedland - nearest hotel: The Esplanade Hotel
salinas - nearest hotel: The Traveler's Hotel
saldanha - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
0,winnemucca,US,40.9730,-117.7357,48,The Overland Hotel
4,bethel,US,41.3712,-73.4140,58,Hampton Inn Danbury
20,dryden,CA,49.7833,-92.7503,46,Holiday Inn Express
50,hermanus,ZA,-34.4187,19.2345,77,Aloe guest house
68,bredasdorp,ZA,-34.5322,20.0403,77,Victoria Hotel
77,lompoc,US,34.6391,-120.4579,73,Red Roof Inn Lompoc
99,kruisfontein,ZA,-34.0033,24.7314,86,No hotel found
105,port lincoln,AU,-34.7333,135.8667,79,Boston Hotel
113,la plata,AR,-34.9215,-57.9545,42,Dazzler by Wyndham La Plata
128,shihezi,CN,44.3000,86.0333,26,石河子宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoEco",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)